In [1]:
#import data set
fbCombinedFileName = "/FileStore/tables/TrendsMarket/facebook_combined.txt"
fbCombinedFile = sc.textFile(fbCombinedFileName)
fbCombinedFile.take(10)

Out[2]: ['0 1', '0 2', '0 3', '0 4', '0 5', '0 6', '0 7', '0 8', '0 9', '0 10']

In [2]:
#retrieve vertex and edges from the data set
def get_vertex1_tuple(entry):
  row = entry.split(' ')
  return int(row[0])

def get_vertex2_tuple(entry):
  row = entry.split(' ')
  return int(row[1])

def get_edge_tuple(entry):
  row = entry.split(' ')
  return int(row[0]),int(row[1])

In [3]:
#create vertex and edges RDD
from pyspark.sql import Row
vertext1RDD = fbCombinedFile.map(get_vertex1_tuple).cache().distinct()
vertext2RDD = fbCombinedFile.map(get_vertex2_tuple).cache().distinct()
vertex1Union2 = vertext1RDD.union(vertext2RDD)
vertexRDD = vertex1Union2.distinct()
vertexCount = vertexRDD.count()
print (vertexCount)
print ('Vertices: {0}'.format(vertexRDD.takeOrdered(5)))
edgesRDD = fbCombinedFile.map(get_edge_tuple).cache()
ecount = edgesRDD.count()
print (ecount)
print ('edges: {0}'.format(edgesRDD.take(5)))

4039
Vertices: [0, 1, 2, 3, 4]
88234
edges: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]

In [4]:
import igraph
from igraph import *

In [5]:
#build igraph with vertices and edges from the dataset
vertices = vertexRDD.collect()
edges = edgesRDD.collect()
g = Graph(vertex_attrs={"label":vertices}, edges=edges, directed=False)

In [6]:
!pip install libcairo2-dev 
!pip install libjpeg-dev 
!pip install libgif-dev
!/home/ubuntu/databricks/python/bin/pip install --user git+https://github.com/pygobject/pycairo.git

#plot(g)

In [7]:
from functools import reduce

#overall dataset analysis on the built graph
print(g.is_connected(mode=STRONG))
print(g.farthest_points(directed=False, unconn=True, weights=None))
nwDiameter = g.diameter(directed=False, unconn=True, weights=None)
print(nwDiameter)
print(g.get_diameter(directed=False, unconn=True, weights=None))
nwBetweeness = g.betweenness(vertices=None, directed=False, cutoff=None, weights=None, nobigint=True)
meanNwBetweeness= reduce(lambda x, y: x + y, nwBetweeness) / len(nwBetweeness)
print(meanNwBetweeness)

True
(687, 3981, 8)
8
[687, 686, 698, 3437, 567, 414, 594, 3980, 3981]
5436.171329537039

In [8]:
#check degree distribution of the network
nwDegrees = g.degree()
meanNwDegree= reduce(lambda x, y: x + y, nwDegrees) / len(nwDegrees)
print(meanNwDegree)
from operator import add
nwDegreesRDD = sc.parallelize(nwDegrees)
counts = nwDegreesRDD.map(lambda x: (x, 1)).reduceByKey(add)
output = counts.collect()

degrees = []
counts = []

for (degree, count) in output:
  print("{0} {1}".format(degree, count))
  degrees.append(degree)
  counts.append(count)

43.69101262688784
8 111
16 82
24 60
32 44
40 27
48 24
56 18
72 15
96 11
64 13
104 6
128 5
120 5
80 8
88 9
152 4
112 7
136 4
160 6
792 1
192 2
176 4
224 1
168 5
184 3
200 1
144 1
17 76
57 23
1 75
9 100
65 20
49 33
25 55
41 29
73 10
97 8
33 35
89 9
81 4
113 8
217 1
137 5
193 3
169 3
105 7
145 3
177 4
185 4
121 6
209 1
201 4
161 4
153 2
129 4
10 95
2 98
26 56
34 43
18 73
42 21
66 22
122 12
58 15
50 25
98 5
170 3
130 6
106 9
82 12
154 4
114 4
74 10
90 7
146 5
234 2
178 5
186 4
226 1
202 2
162 2
138 2
194 1
210 1
347 1
11 81
19 72
3 93
43 29
27 49
59 11
51 20
99 11
115 10
107 9
83 17
67 13
91 9
35 36
147 6
75 8
123 11
139 5
131 6
171 3
211 1
179 3
235 1
187 4
155 5
195 4
755 1
203 2
163 1
291 1
547 1
20 63
68 16
44 21
12 82
4 99
76 15
36 43
28 37
60 18
84 12
52 19
100 10
132 4
156 6
124 9
92 10
148 3
220 1
108 6
172 3
204 1
164 4
188 4
140 3
116 6
180 2
196 2
13 79
69 14
5 93
21 52
1045 1
37 43
133 3
45 19
229 1
61 18
77 10
53 15
29 38
85 10
109 9
149 1
165 5
205 4
141 7
117 9
173 6
125 5
93 11
189 3
245 1
101 4
181 2
221 1
157 1
197 3
6 98
22 63
78 11
14 87
62 16
38 44
46 24
30 40
102 9
94 7
54 23
126 6
166 4
142 7
70 17
198 5
158 5
110 4
182 5
190 6
150 2
254 1
134 3
86 9
174 2
222 1
294 1
31 38
15 106
23 53
7 98
63 23
47 24
39 29
55 23
71 18
159 3
231 1
87 5
79 16
167 2
103 6
95 9
183 3
135 7
175 2
111 1
151 5
127 4
191 4
119 4
223 1
199 2
207 3

In [9]:
# Node degree vs frequency graph
import matplotlib.pyplot as plt 
from matplotlib.pyplot import MultipleLocator

plt.bar(degrees, counts)
x_major_locator=MultipleLocator(20)
ax=plt.gca()
ax.xaxis.set_major_locator(x_major_locator)
plt.xlim(-0.5, 300)
plt.ylim(0, 120)
plt.xlabel('Node Degree')
plt.ylabel('Frequency')
plt.title('Frequency by Node Degree')
plt.show()
display()

In [10]:
#identify insignificant nodes
island_list = []
for v in vertices:
  friends_list = g.neighbors(vertex=v, mode=ALL)
  if (len(friends_list) < 2):
    island_list.append(v)
print(set(island_list))

island_degree_list=[]
for i in island_list:
  island_degree_list.append(g.degree(i))
print(set(island_degree_list))

#remove island nodes from the graph 
g.delete_vertices(island_list)
newVertices = []
newEdges = []
for v in g.vs:
    newVertices.append(v["label"])
for e in g.es:
    newEdges.append(e.tuple) 
print(len(set(vertices)))    
print(len(set(island_list)))    
print(len(set(newVertices)))    

{3974, 2569, 11, 12, 3853, 15, 3856, 3984, 3729, 18, 911, 2195, 918, 1560, 2457, 2842, 668, 3230, 287, 2079, 801, 674, 292, 2596, 3748, 37, 4008, 550, 1834, 2470, 4010, 1581, 43, 4015, 692, 1206, 4022, 4024, 1466, 447, 3650, 4035, 1096, 585, 74, 335, 209, 210, 3451, 3798, 215, 3031, 602, 2269, 1119, 608, 607, 3935, 613, 3709, 358, 1386, 875, 3820, 3183, 624, 114, 3570, 883, 1145, 891, 892, 3453, 638, 3071}
{1}
4039
75
3964

In [11]:
#identify significant nodes
core_node_list = []
core_degree_list = []
for v in g.vs:
  v_degree = g.degree(v)
  if(v_degree > 300): 
    core_node_list.append(v.index)
    core_degree_list.append(v_degree)
print(set(core_node_list))
mean_core_degree = reduce(lambda x, y: x + y, core_degree_list) / len(core_degree_list)
print(mean_core_degree)
#sub graph focussing on node "0" which was identified as significant node
node0_friends_list = g.neighbors(vertex=0, mode=ALL)
freinds_of_friends = g.neighborhood(vertices=0, order=2, mode=ALL)
print(len(node0_friends_list))
print(len(freinds_of_friends))

node0_friends_list.append(0)
node0_alters = []
node0_graph = g.subgraph(node0_friends_list, implementation = "auto")

for e in node0_graph.es:
    print(e.tuple)
    node0_alters.append(e.tuple)

{0, 100, 1642, 1869, 3382}
687.2
333
1494
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
(0, 11)
(0, 12)
(0, 13)
(0, 14)
(0, 15)
(0, 16)
(0, 17)
(0, 18)
(0, 19)
(0, 20)
(0, 21)
(0, 22)
(0, 23)
(0, 24)
(0, 25)
(0, 26)
(0, 27)
(0, 28)
(0, 29)
(0, 30)
(0, 31)
(0, 32)
(0, 33)
(0, 34)
(0, 35)
(0, 36)
(0, 37)
(0, 38)
(0, 39)
(0, 40)
(0, 41)
(0, 42)
(0, 43)
(0, 44)
(0, 45)
(0, 46)
(0, 47)
(0, 48)
(0, 49)
(0, 50)
(0, 51)
(0, 52)
(0, 53)
(0, 54)
(0, 55)
(0, 56)
(0, 57)
(0, 58)
(0, 59)
(0, 60)
(0, 61)
(0, 62)
(0, 63)
(0, 64)
(0, 65)
(0, 66)
(0, 67)
(0, 68)
(0, 69)
(0, 70)
(0, 71)
(0, 72)
(0, 73)
(0, 74)
(0, 75)
(0, 76)
(0, 77)
(0, 78)
(0, 79)
(0, 80)
(0, 81)
(0, 82)
(0, 83)
(0, 84)
(0, 85)
(0, 86)
(0, 87)
(0, 88)
(0, 89)
(0, 90)
(0, 91)
(0, 92)
(0, 93)
(0, 94)
(0, 95)
(0, 96)
(0, 97)
(0, 98)
(0, 99)
(0, 100)
(0, 101)
(0, 102)
(0, 103)
(0, 104)
(0, 105)
(0, 106)
(0, 107)
(0, 108)
(0, 109)
(0, 110)
(0, 111)
(0, 112)
(0, 113)
(0, 114)
(0, 115)
(0, 116)
(0, 117)
(0, 118)
(0, 119)
(0, 120)
(0, 121)
(0, 122)
(0, 123)
(0, 124)
(0, 125)
(0, 126)
(0, 127)
(0, 128)
(0, 129)
(0, 130)
(0, 131)
(0, 132)
(0, 133)
(0, 134)
(0, 135)
(0, 136)
(0, 137)
(0, 138)
(0, 139)
(0, 140)
(0, 141)
(0, 142)
(0, 143)
(0, 144)
(0, 145)
(0, 146)
(0, 147)
(0, 148)
(0, 149)
(0, 150)
(0, 151)
(0, 152)
(0, 153)
(0, 154)
(0, 155)
(0, 156)
(0, 157)
(0, 158)
(0, 159)
(0, 160)
(0, 161)
(0, 162)
(0, 163)
(0, 164)
(0, 165)
(0, 166)
(0, 167)
(0, 168)
(0, 169)
(0, 170)
(0, 171)
(0, 172)
(0, 173)
(0, 174)
(0, 175)
(0, 176)
(0, 177)
(0, 178)
(0, 179)
(0, 180)
(0, 181)
(0, 182)
(0, 183)
(0, 184)
(0, 185)
(0, 186)
(0, 187)
(0, 188)
(0, 189)
(0, 190)
(0, 191)
(0, 192)
(0, 193)
(0, 194)
(0, 195)
(0, 196)
(0, 197)
(0, 198)
(0, 199)
(0, 200)
(0, 201)
(0, 202)
(0, 203)
(0, 204)
(0, 205)
(0, 206)
(0, 207)
(0, 208)
(0, 209)
(0, 210)
(0, 211)
(0, 212)
(0, 213)
(0, 214)
(0, 215)
(0, 216)
(0, 217)
(0, 218)
(0, 219)
(0, 220)
(0, 221)
(0, 222)
(0, 223)
(0, 224)
(0, 225)
(0, 226)
(0, 227)
(0, 228)
(0, 229)
(0, 230)
(0, 231)
(0, 232)
(0, 233)
(0, 234)
(0, 235)
(0, 236)
(0, 237)
(0, 238)
(0, 239)
(0, 240)
(0, 241)
(0, 242)
(0, 243)
(0, 244)
(0, 245)
(0, 246)
(0, 247)
(0, 248)
(0, 249)
(0, 250)
(0, 251)
(0, 252)
(0, 253)
(0, 254)
(0, 255)
(0, 256)
(0, 257)
(0, 258)
(0, 259)
(0, 260)
(0, 261)
(0, 262)
(0, 263)
(0, 264)
(0, 265)
(0, 266)
(0, 267)
(0, 268)
(0, 269)
(0, 270)
(0, 271)
(0, 272)
(0, 273)
(0, 274)
(0, 275)
(0, 276)
(0, 277)
(0, 278)
(0, 279)
(0, 280)
(0, 281)
(0, 282)
(0, 283)
(0, 284)
(0, 285)
(0, 286)
(0, 287)
(0, 288)
(0, 289)
(0, 290)
(0, 291)
(0, 292)
(0, 293)
(0, 294)
(0, 295)
(0, 296)
(0, 297)
(0, 298)
(0, 299)
(0, 300)
(0, 301)
(0, 302)
(0, 303)
(0, 304)
(0, 305)
(0, 306)
(0, 307)
(0, 308)
(0, 309)
(0, 310)
(0, 311)
(0, 312)
(0, 313)
(0, 314)
(0, 315)
(0, 316)
(0, 317)
(0, 318)
(0, 319)
(0, 320)
(0, 321)
(0, 322)
(0, 323)
(0, 324)
(0, 325)
(0, 326)
(0, 327)
(0, 328)
(0, 329)
(0, 330)
(0, 331)
(0, 332)
(0, 333)
(1, 42)
(1, 47)
(1, 48)
(1, 67)
(1, 81)
(1, 85)
(1, 111)
(1, 118)
(1, 125)
(1, 186)
(1, 225)
(1, 269)
(1, 286)
(1, 302)
(1, 309)
(1, 332)
(2, 16)
(2, 107)
(2, 108)
(2, 141)
(2, 215)
(2, 299)
(2, 313)
(2, 320)
(2, 329)
(3, 9)
(3, 21)
(3, 22)
(3, 61)
(3, 66)
(3, 78)
(3, 114)
(3, 134)
(3, 162)
(3, 180)
(3, 192)
(3, 217)
(3, 263)
(3, 269)
(3, 272)
(3, 310)
(4, 71)
(4, 144)
(4, 173)
(4, 187)
(4, 207)
(4, 262)
(4, 264)
(4, 293)
(4, 315)
(5, 80)
(5, 114)
(5, 148)
(5, 150)
(5, 161)
(5, 172)
(5, 179)
(5, 196)
(5, 203)
(5, 224)
(5, 302)
(5, 303)
(6, 82)
(6, 88)
(6, 139)
(6, 208)
(6, 306)
(7, 18)
(7, 27)
(7, 33)
(7, 59)
(7, 80)
(7, 96)
(7, 121)
(7, 128)
(7, 160)
(7, 203)
(7, 235)
(7, 279)
(7, 291)
(7, 295)
(7, 302)
(7, 309)
(7, 325)
(7, 326)
(7, 333)
(8, 84)
(8, 103)
(8, 185)
(8, 193)
(8, 234)
(8, 248)
(8, 253)
(9, 17)
(9, 21)
(9, 22)
(9, 26)
(9, 50)
(9, 60)
(9, 61)
(9, 63)
(9, 66)
(9, 68)
(9, 72)
(9, 78)
(9, 98)
(9, 106)
(9, 111)
(9, 114)
(9, 120)
(9, 125)
(9, 126)
(9, 133)
(9, 134)
(9, 140)
(9, 148)
(9, 153)
(9, 161)
(9, 162)
(9, 168)
(9, 177)
(9, 178)
(9, 180)


In [12]:
node0_graph.write_svg('temp.svg')

In [13]:
%fs ls 

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/ml/,ml/,0
dbfs:/tmp/,tmp/,0


In [14]:
#identify cliques on the subgraph
cliques_0 = node0_graph.maximal_cliques(min =2 , max =10)
print(cliques_0)

[(223, 0, 228), (152, 0, 249), (137, 0, 171), (37, 0, 29), (171, 0, 83), (205, 0, 232), (303, 0, 5), (41, 0, 288), (275, 0, 74), (43, 0, 184), (43, 0, 230), (43, 0, 244), (175, 0, 125), (197, 0, 184), (46, 0, 184), (145, 0, 269), (184, 0, 107), (190, 0, 158), (271, 0, 233), (268, 0, 104), (74, 0, 258), (146, 0, 208), (256, 0, 19), (64, 0, 219), (245, 0, 222), (288, 0, 311), (292, 0, 299), (30, 0, 165), (31, 0, 135), (130, 0, 15), (174, 0, 90), (174, 0, 242), (112, 0, 128), (112, 0, 236), (112, 0, 266), (105, 0, 166, 15), (100, 0, 163, 52), (90, 0, 219), (322, 0, 81, 21), (281, 0, 70), (281, 0, 266), (280, 0, 166, 15), (259, 0, 21, 69), (259, 0, 21, 195), (308, 0, 167, 135), (242, 0, 219), (236, 0, 25), (229, 0, 19), (229, 0, 211), (217, 0, 272, 3), (199, 0, 309, 50), (194, 0, 165, 158), (194, 0, 316), (52, 0, 258), (183, 0, 122, 47), (55, 0, 19), (55, 0, 185), (116, 0, 19), (116, 0, 149), (116, 0, 162), (165, 0, 73), (165, 0, 155), (156, 0, 81), (156, 0, 274), (65, 0, 294, 219), (149, 0, 147), (147, 0, 19), (294, 0, 35), (117, 0, 111), (117, 0, 161, 128), (193, 0, 248, 84, 8), (219, 0, 19), (219, 0, 204, 36), (251, 0, 299, 209, 107), (182, 0, 19), (182, 0, 89), (182, 0, 218), (272, 0, 21, 3), (272, 0, 110), (198, 0, 113), (198, 0, 170), (198, 0, 249), (276, 0, 191, 21), (276, 0, 191, 302), (277, 0, 204, 15), (277, 0, 204, 129), (258, 0, 111, 93), (258, 0, 111, 316), (314, 0, 306, 88, 82), (185, 0, 248, 84, 8), (82, 0, 306, 15), (82, 0, 306, 139, 6, 88), (82, 0, 306, 139, 6, 208), (8, 0, 248, 84, 234), (8, 0, 248, 84, 253, 103), (23, 0, 316, 111, 48), (23, 0, 316, 111, 311), (40, 0, 267, 169, 167, 135), (70, 0, 220, 109), (70, 0, 220, 319, 35), (109, 0, 33), (109, 0, 269, 220), (109, 0, 274), (144, 0, 264, 173, 4), (144, 0, 325), (155, 0, 142, 111, 93), (155, 0, 238), (218, 0, 140, 111, 102), (218, 0, 321), (224, 0, 114, 17), (224, 0, 114, 148, 5), (234, 0, 19), (239, 0, 241, 53), (239, 0, 241, 309, 162), (38, 0, 329, 299, 104, 36, 16), (326, 0, 235, 7, 203), (326, 0, 235, 7, 325), (326, 0, 311, 203), (58, 0, 206, 181, 142, 111, 93), (58, 0, 206, 181, 142, 111, 163), (249, 0, 309, 47), (249, 0, 309, 118), (249, 0, 309, 162), (249, 0, 309, 325), (95, 0, 283, 252, 216, 167, 92), (28, 0, 323, 297, 159, 129, 86), (159, 0, 323, 297, 129, 232), (283, 0, 252, 216, 167, 92, 62), (206, 0, 142, 111, 311), (118, 0, 81, 42, 1, 67), (118, 0, 81, 42, 1, 309), (86, 0, 323, 297, 129, 104, 36), (57, 0, 328, 300, 260, 250, 50), (93, 0, 111, 310), (142, 0, 111, 120), (142, 0, 111, 316, 311), (323, 0, 297, 129, 36, 232), (79, 0, 267, 216, 214, 169, 167), (232, 0, 129, 36, 299), (169, 0, 216, 214, 135, 92, 62, 167), (169, 0, 216, 214, 135, 92, 267, 123), (169, 0, 216, 214, 135, 92, 267, 167), (297, 0, 36, 14, 129, 104), (297, 0, 36, 143, 24, 12), (226, 0, 76, 45, 19), (226, 0, 76, 45, 77, 27, 21), (216, 0, 167, 15), (216, 0, 167, 92, 19), (167, 0, 266), (48, 0, 47, 42, 1), (48, 0, 111, 42, 1), (48, 0, 111, 300), (121, 0, 325, 160, 7, 279, 27), (121, 0, 325, 160, 7, 333), (287, 0, 332, 87, 73, 286), (287, 0, 332, 87, 73, 307), (287, 0, 332, 122), (15, 0, 299, 204, 107, 36, 14), (15, 0, 299, 204, 107, 36, 329), (304, 0, 47, 94), (304, 0, 47, 138), (304, 0, 150, 124, 35), (143, 0, 299, 136, 36, 24, 12), (324, 0, 195, 25), (324, 0, 195, 228), (324, 0, 195, 295), (324, 0, 241, 25), (324, 0, 241, 228), (324, 0, 241, 295, 27), (19, 0, 136, 108), (298, 0, 254, 237), (298, 0, 254, 312), (298, 0, 266, 124), (298, 0, 266, 237), (298, 0, 266, 312), (298, 0, 330), (200, 0, 266, 180, 274, 111), (200, 0, 266, 180, 279), (200, 0, 266, 319, 274, 111), (200, 0, 266, 319, 279), (333, 0, 160, 7, 291), (333, 0, 160, 7, 325, 18), (210, 0, 309, 178, 21, 274, 50), (210, 0, 309, 178, 290, 39), (210, 0, 309, 178, 290, 274, 50), (54, 0, 50, 274, 228, 300), (54, 0, 50, 274, 228, 309), (54, 0, 50, 279), (54, 0, 150, 279), (54, 0, 150, 309), (33, 0, 133), (33, 0, 237, 319), (33, 0, 309, 170), (33, 0, 309, 325, 7), (154, 0, 329, 141, 313, 299, 16), (154, 0, 329, 141, 313, 299, 1

In [15]:
#community detection with centrality based approach using edge betweeness
communities = node0_graph.community_edge_betweenness(directed=False)
clusters = communities.as_clustering()
print(clusters.modularity)
print(clusters)

# timetaken: 33.09s

0.35597592496788777
Clustering with 334 elements and 27 clusters
[ 0] 0, 3, 5, 7, 9, 10, 11, 13, 17, 18, 21, 22, 23, 25, 26, 27, 33, 34, 35,
 39, 44, 48, 49, 50, 53, 54, 56, 57, 59, 60, 61, 63, 66, 68, 69, 70, 72,
 75, 77, 78, 80, 81, 89, 91, 96, 97, 98, 99, 102, 106, 109, 110, 111, 113,
 114, 115, 117, 120, 121, 124, 125, 126, 128, 133, 134, 138, 140, 145,
 148, 150, 153, 156, 157, 160, 161, 162, 164, 168, 170, 177, 178, 180,
 182, 191, 192, 195, 198, 199, 200, 201, 202, 203, 210, 211, 212, 213,
 218, 220, 221, 223, 224, 225, 227, 228, 235, 237, 239, 241, 246, 247,
 249, 250, 254, 257, 259, 260, 261, 263, 265, 266, 269, 274, 276, 278,
 279, 281, 282, 284, 285, 290, 291, 295, 298, 300, 301, 302, 304, 305,
 309, 310, 311, 312, 316, 318, 319, 321, 322, 324, 325, 326, 327, 328,
 330, 331, 333
[ 1] 1, 20, 42, 47, 51, 67, 73, 85, 87, 94, 118, 122, 155, 172, 179, 183, 186,
 188, 196, 231, 238, 243, 255, 286, 287, 289, 317, 332
[ 2] 2, 12, 14, 16, 24, 28, 36, 38, 86, 104, 107, 108, 129, 132, 136, 141,
 143, 154, 159, 204, 205, 209, 215, 232, 251, 268, 277, 292, 297, 299,
 313, 320, 323, 329
[ 3] 4, 71, 144, 173, 187, 207, 262, 264, 293, 315
[ 4] 6, 82, 88, 139, 146, 208, 306, 314
[ 5] 8, 55, 84, 103, 185, 193, 234, 248, 253
[ 6] 15, 105, 130, 166, 280
[ 7] 19, 45, 76, 226, 229, 256
[ 8] 29, 37
[ 9] 30, 158, 165, 190, 194
[10] 31, 40, 62, 79, 92, 95, 123, 135, 167, 169, 214, 216, 252, 267, 283, 308
[11] 32, 101, 119, 127, 131, 151, 176, 189, 240, 270, 273, 296, 307
[12] 41, 288
[13] 43, 230, 244
[14] 46, 184, 197
[15] 52, 58, 93, 100, 142, 163, 181, 206, 258
[16] 64, 65, 90, 174, 219, 242, 294
[17] 74, 275
[18] 83, 137, 171
[19] 112, 236
[20] 116, 147, 149
[21] 152
[22] 175
[23] 217, 272
[24] 222, 245
[25] 233, 271
[26] 303

In [16]:
#community detection using fast greedy algorithm
fastGreedy = node0_graph.community_fastgreedy()
FGcluster = fastGreedy.as_clustering()
print(FGcluster.modularity)
print(FGcluster)

# timetaken: 0.05s

0.41064905609901475
Clustering with 334 elements and 8 clusters
[0] 0, 1, 5, 20, 23, 26, 29, 30, 37, 41, 42, 43, 46, 47, 48, 51, 52, 58, 64,
 65, 67, 73, 74, 81, 83, 85, 87, 89, 90, 93, 94, 99, 100, 111, 117, 118,
 122, 137, 138, 142, 145, 152, 155, 156, 158, 163, 165, 170, 171, 172, 174,
 175, 179, 181, 182, 183, 184, 186, 188, 190, 194, 196, 197, 198, 200, 203,
 206, 217, 218, 219, 222, 223, 224, 230, 231, 233, 238, 242, 243, 244, 245,
 249, 255, 258, 271, 272, 275, 286, 287, 288, 289, 294, 303, 304, 307, 311,
 316, 317, 322, 326, 332
[1] 2, 12, 14, 15, 16, 24, 28, 36, 38, 86, 104, 105, 107, 108, 129, 130, 132,
 136, 141, 143, 154, 159, 166, 204, 205, 209, 215, 232, 251, 268, 277, 280,
 292, 297, 299, 313, 320, 323, 329
[2] 3, 9, 10, 11, 13, 17, 21, 22, 25, 34, 35, 39, 44, 49, 50, 53, 54, 56, 57,
 59, 60, 61, 63, 66, 68, 69, 72, 75, 78, 91, 96, 97, 98, 102, 106, 110,
 113, 114, 115, 120, 124, 125, 126, 133, 134, 140, 148, 150, 153, 157, 161,
 162, 164, 168, 177, 178, 180, 191, 192, 195, 199, 201, 202, 210, 211, 212,
 213, 220, 221, 225, 227, 228, 237, 239, 241, 246, 247, 250, 254, 257, 259,
 260, 261, 263, 265, 266, 269, 274, 276, 278, 279, 282, 284, 285, 290, 291,
 298, 300, 301, 302, 305, 309, 310, 312, 318, 319, 321, 327, 328, 330, 331
[3] 4, 71, 144, 173, 187, 207, 262, 264, 293, 315
[4] 6, 82, 88, 139, 146, 208, 306, 314
[5] 7, 18, 27, 33, 45, 70, 76, 77, 80, 109, 112, 121, 128, 160, 226, 235, 236,
 281, 295, 324, 325, 333
[6] 8, 19, 31, 40, 55, 62, 79, 84, 92, 95, 103, 116, 123, 135, 147, 149, 167,
 169, 185, 193, 214, 216, 229, 234, 248, 252, 253, 256, 267, 283, 308
[7] 32, 101, 119, 127, 131, 151, 176, 189, 240, 270, 273, 296

In [17]:
#community detection using walk trap algorithm
walkTrap = node0_graph.community_walktrap() 
WTcluster = walkTrap.as_clustering()
print(WTcluster.modularity)
print(WTcluster)

#timetaken: 0.08s

0.3516264145728796
Clustering with 334 elements and 40 clusters
[ 0] 0, 5, 17, 23, 26, 33, 39, 48, 70, 109, 111, 114, 117, 124, 128, 138, 145,
 148, 150, 170, 175, 199, 203, 210, 218, 223, 224, 261, 281, 290, 295,
 302, 303, 304, 309, 311, 316, 331
[ 1] 1, 42, 67, 81, 118, 122, 156, 183, 322
[ 2] 2, 12, 16, 24, 107, 108, 132, 136, 141, 154, 215, 292, 299, 313, 320, 329
[ 3] 3, 9, 10, 11, 13, 21, 22, 25, 27, 34, 35, 44, 49, 50, 53, 54, 56, 57, 59,
 60, 61, 63, 66, 68, 72, 75, 77, 78, 89, 91, 96, 97, 98, 99, 102, 106,
 110, 113, 115, 120, 125, 126, 133, 134, 140, 153, 157, 161, 162, 164,
 168, 177, 178, 180, 191, 192, 195, 200, 201, 202, 211, 212, 213, 220,
 221, 225, 227, 228, 237, 239, 241, 246, 247, 250, 254, 257, 260, 263,
 265, 266, 269, 274, 276, 278, 279, 282, 284, 285, 291, 298, 300, 301,
 305, 310, 312, 318, 319, 321, 324, 327, 328, 330
[ 4] 4, 71, 144, 173, 187, 207, 262, 264, 293, 315
[ 5] 6, 82, 88, 139, 208, 306, 314
[ 6] 7, 18, 80, 121, 160, 235, 325, 326, 333
[ 7] 8, 84, 103, 185, 193, 234, 248, 253
[ 8] 14, 36, 38, 104, 143, 204, 268, 277
[ 9] 15, 130
[10] 19, 45, 76, 182, 226, 229, 256
[11] 20, 47, 51, 73, 85, 87, 94, 172, 179, 186, 188, 196, 231, 238, 243, 255,
 286, 287, 289, 317, 332
[12] 28, 86, 129, 159, 232, 297, 323
[13] 29, 37
[14] 30, 165, 194
[15] 31
[16] 32, 101, 119, 127, 131, 151, 176, 189, 240, 270, 273, 296, 307
[17] 40, 62, 79, 92, 95, 123, 135, 167, 169, 214, 216, 252, 267, 283, 308
[18] 41, 288
[19] 43, 230, 244
[20] 46, 184, 197
[21] 52, 100
[22] 55
[23] 58, 93, 142, 155, 163, 181, 206, 258
[24] 64, 65, 219, 294
[25] 69, 259
[26] 74, 275
[27] 83, 137, 171
[28] 90, 174, 242
[29] 105, 166, 280
[30] 112, 236
[31] 116, 147, 149
[32] 146
[33] 152, 198, 249
[34] 158, 190
[35] 205
[36] 209, 251
[37] 217, 272
[38] 222, 245
[39] 233, 271

In [18]:
#community detection using info map algorithm
infoMap = node0_graph.community_infomap()
print(infoMap.modularity)
print(infoMap.as_cover())

# timetaken: 0.65s

0.38573930796331013
Cover with 24 clusters
[ 0] 3, 5, 7, 9, 10, 11, 13, 17, 18, 21, 22, 23, 25, 26, 27, 33, 34, 35, 39,
 44, 49, 50, 53, 54, 56, 57, 59, 60, 61, 63, 66, 68, 70, 72, 75, 78, 80,
 89, 91, 96, 97, 98, 99, 102, 106, 109, 110, 111, 113, 114, 115, 117, 120,
 121, 124, 125, 126, 128, 133, 134, 138, 140, 148, 150, 153, 157, 160,
 161, 162, 164, 168, 170, 177, 178, 180, 191, 192, 195, 199, 200, 201,
 202, 203, 210, 211, 212, 213, 218, 220, 221, 224, 225, 227, 228, 235,
 237, 239, 241, 246, 247, 250, 254, 257, 260, 261, 263, 265, 266, 269,
 274, 276, 278, 279, 281, 282, 284, 285, 290, 291, 295, 298, 300, 301,
 302, 304, 305, 309, 310, 311, 312, 316, 318, 319, 321, 324, 325, 326,
 327, 328, 330, 331, 333
[ 1] 2, 12, 14, 16, 24, 28, 36, 38, 86, 104, 107, 108, 129, 132, 136, 141,
 143, 154, 159, 204, 209, 215, 232, 251, 277, 297, 299, 313, 320, 323, 329
[ 2] 0, 69, 145, 175, 205, 217, 223, 259, 268, 272, 292, 303
[ 3] 20, 47, 51, 73, 85, 87, 94, 172, 179, 186, 188, 196, 231, 238, 243, 255,
 286, 287, 289, 317, 332
[ 4] 31, 40, 62, 79, 92, 95, 123, 135, 167, 169, 214, 216, 252, 267, 283, 308
[ 5] 32, 101, 119, 127, 131, 151, 176, 189, 240, 270, 273, 296, 307
[ 6] 4, 71, 144, 173, 187, 207, 262, 264, 293, 315
[ 7] 1, 42, 48, 67, 81, 118, 122, 156, 183, 322
[ 8] 19, 45, 76, 77, 116, 147, 149, 182, 226, 229, 256
[ 9] 52, 58, 93, 100, 142, 155, 163, 181, 206, 258
[10] 8, 55, 84, 103, 185, 193, 234, 248, 253
[11] 6, 82, 88, 139, 146, 208, 306, 314
[12] 64, 65, 90, 174, 219, 242, 294
[13] 15, 105, 130, 166, 280
[14] 43, 46, 184, 197, 230, 244
[15] 30, 158, 165, 190, 194
[16] 152, 198, 249
[17] 83, 137, 171
[18] 74, 275
[19] 112, 236
[20] 233, 271
[21] 222, 245
[22] 29, 37
[23] 41, 288

In [19]:
#Part 2 - Friend Recommendation based on clusters detected
#Extract tuples from dataset
def returnTuple(entry):
  row = entry.split(' ')
  return int(row[0]),int(row[1]),-1

egoRDD = fbCombinedFile.map(returnTuple)

In [20]:
#Detect no.of.mutual friends for any two pairs of nodes from the graph 
mutualFriends=[]
def generate(x):
  toNodes=[]
  for row in egoRDD.collect():
    if row[0]==x:
      toNodes.append(row[1])
  for i in range(0,len(toNodes)-1):
    mutualFriends.append([toNodes[i],toNodes[i+1],1])
    
prev = -1
  
for row in egoRDD.collect():
  if row[0]!=prev:
    generate(row[0])
  prev=row[0]
  
def predict(entry):
  return (entry[0],entry[1]),entry[2]
  
mutualFriendsRDD =sc.parallelize(mutualFriends)
prediction=mutualFriendsRDD.map(predict)

PredictionRDD = prediction.reduceByKey(lambda a, b: a + b)
sortedRdd=PredictionRDD.sortBy(lambda a: -a[1])
print(sortedRdd.collect())

[((2624, 2625), 163), ((2602, 2604), 163), ((2607, 2611), 161), ((2601, 2602), 161), ((2586, 2590), 155), ((1800, 1804), 153), ((2600, 2601), 149), ((2625, 2630), 147), ((2593, 2600), 142), ((2615, 2619), 137), ((2611, 2615), 136), ((2654, 2655), 134), ((2559, 2560), 132), ((2560, 2561), 122), ((2542, 2543), 122), ((1799, 1800), 121), ((1827, 1833), 120), ((2507, 2520), 114), ((2623, 2624), 114), ((2409, 2410), 112), ((2564, 2573), 112), ((2410, 2414), 112), ((1833, 1835), 111), ((2549, 2550), 110), ((2638, 2646), 105), ((2539, 2542), 105), ((2604, 2607), 104), ((2550, 2551), 102), ((2500, 2504), 101), ((1584, 1589), 99), ((2630, 2631), 99), ((1886, 1888), 98), ((2646, 2654), 98), ((1746, 1750), 97), ((2492, 2495), 96), ((2546, 2549), 96), ((2631, 2638), 96), ((2520, 2521), 94), ((2579, 2586), 94), ((2619, 2623), 93), ((2510, 2511), 93), ((2423, 2428), 92), ((2556, 2559), 92), ((2578, 2579), 92), ((2464, 2467), 91), ((2323, 2324), 91), ((2339, 2340), 91), ((2340, 2347), 91), ((2369, 2370), 90), ((2408, 2409), 90), ((2354, 2356), 90), ((2376, 2381), 89), ((2395, 2404), 89), ((2597, 2598), 88), ((1662, 1663), 88), ((2363, 2369), 87), ((2521, 2526), 87), ((2543, 2546), 87), ((1612, 1613), 86), ((2266, 2271), 86), ((2506, 2507), 86), ((2552, 2553), 85), ((2324, 2326), 85), ((2374, 2376), 84), ((2370, 2374), 83), ((2563, 2564), 83), ((2590, 2591), 81), ((1621, 1622), 80), ((2308, 2309), 80), ((2356, 2359), 80), ((2590, 2593), 80), ((2271, 2275), 78), ((2229, 2233), 78), ((1714, 1717), 78), ((2430, 2433), 78), ((1610, 1612), 78), ((1835, 1839), 77), ((2359, 2363), 77), ((2446, 2460), 77), ((2591, 2593), 77), ((2433, 2446), 76), ((2347, 2348), 75), ((2461, 2463), 74), ((2561, 2563), 74), ((2482, 2484), 74), ((2240, 2244), 74), ((2555, 2567), 73), ((2509, 2510), 73), ((1750, 1752), 73), ((1839, 1842), 73), ((2575, 2578), 73), ((2220, 2229), 72), ((2643, 2649), 72), ((2414, 2418), 72), ((2218, 2220), 72), ((1804, 1809), 72), ((1663, 1665), 72), ((2489, 2492), 72), ((2526, 2532), 71), ((2468, 2469), 70), ((2573, 2574), 69), ((1888, 1898), 69), ((2551, 2552), 69), ((2484, 2485), 68), ((2504, 2506), 68), ((1879, 1888), 68), ((2275, 2276), 68), ((2201, 2206), 68), ((2290, 2299), 68), ((2348, 2352), 68), ((2617, 2629), 67), ((1707, 1712), 67), ((2553, 2554), 67), ((2462, 2464), 67), ((1791, 1793), 67), ((1669, 1675), 67), ((2333, 2336), 67), ((2418, 2423), 66), ((2642, 2643), 66), ((2381, 2386), 66), ((1879, 1886), 65), ((2282, 2283), 65), ((2606, 2607), 65), ((2309, 2323), 65), ((1868, 1879), 64), ((2467, 2468), 64), ((2329, 2331), 63), ((2347, 2351), 62), ((2582, 2588), 62), ((1826, 1827), 62), ((1741, 1746), 62), ((2307, 2308), 62), ((2616, 2617), 61), ((2244, 2253), 61), ((2326, 2329), 61), ((2336, 2338), 61), ((2604, 2606), 61), ((2511, 2512), 61), ((2331, 2334), 61), ((1712, 1714), 60), ((2276, 2278), 60), ((2574, 2575), 60), ((2485, 2489), 59), ((2451, 2458), 59), ((2460, 2462), 59), ((2327, 2328), 59), ((1888, 1891), 58), ((2499, 2500), 58), ((1783, 1786), 58), ((2508, 2509), 57), ((2200, 2201), 57), ((2261, 2266), 57), ((2184, 2188), 57), ((2289, 2292), 57), ((1665, 1668), 57), ((1376, 1377), 56), ((2332, 2333), 56), ((2334, 2339), 56), ((1551, 1554), 56), ((1554, 1557), 55), ((1420, 1431), 55), ((2629, 2642), 55), ((1757, 1761), 54), ((1730, 1735), 54), ((1813, 1816), 54), ((2592, 2594), 53), ((1390, 1391), 53), ((1717, 1721), 52), ((2495, 2500), 52), ((2619, 2624), 52), ((1898, 1902), 52), ((2458, 2461), 52), ((2188, 2190), 52), ((2190, 2200), 52), ((2216, 2218), 52), ((2428, 2430), 52), ((2407, 2408), 52), ((2428, 2429), 51), ((1761, 1768), 51), ((1735, 1736), 51), ((1823, 1826), 51), ((2253, 2257), 50), ((3417, 3426), 50), ((1557, 1559), 50), ((2377, 2384), 50), ((2460, 2464), 50), ((1559, 1563), 49), ((1823, 1827), 49), ((2543, 2555), 49), ((1707, 1714), 49), ((2404, 2407), 49), ((2140, 2142), 49), ((3339, 3348), 49), ((1833, 1839), 49), ((2553, 2556), 49), ((2469, 2477), 48), ((2554, 2556), 48), ((1622, 1623), 48), ((2589, 25

In [21]:
#Select one user for whom friend suggestion has to be made
fromuser=115
#Filter mutual friend list for the selected user
suggestions_115_1 = sortedRdd.filter(lambda x:x[0][0]==fromuser).map(lambda x:(x[0][1],x[1]))
suggestions_115_2 = sortedRdd.filter(lambda x:x[0][1]==fromuser).map(lambda x:(x[0][0],x[1]))
suggestions_115 = suggestions_115_1.union(suggestions_115_2)
suggestions_115_sorted = suggestions_115.sortBy(lambda x:-x[1])
suggestions_115_RDD = suggestions_115_sorted.map(lambda x:x[0])
print(suggestions_115_RDD.collect())

[116, 111, 41, 138, 149, 112, 114, 20]

In [22]:
#Get all friends of user 115
friends_115_1= egoRDD.filter(lambda x:x[0]==fromuser).map(lambda x:x[1])
friends_115_2= egoRDD.filter(lambda x:x[1]==fromuser).map(lambda x:x[0])
friends_115 = friends_115_1.union(friends_115_2)
print(friends_115.collect())

[116, 137, 140, 144, 149, 192, 214, 220, 226, 262, 312, 326, 343, 0, 2, 14, 17, 19, 20, 28, 41]

In [23]:
#Get all non friends of user 115
already_friends = suggestions_115_RDD.intersection(friends_115)
suggestions = suggestions_115_RDD.subtract(already_friends)
print(suggestions.collect())

[111, 112, 114, 138]

In [24]:
#Narrow down suggestion based on communities
#Communities detected by fastgreedy is opted because of better modularity
suggestion_list = suggestions.collect()
community_based_suggestion=[]
for cluster_index in range(8):
  for member in suggestion_list:
    if member in FGcluster[cluster_index] and 115 in FGcluster[cluster_index]:
      community_based_suggestion.append(member)

print(community_based_suggestion)

[114]